In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
import numpy as np
from scipy.signal import wiener
from tqdm import tqdm
import pandas as pd
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim

sys.path.append(str(Path("..").resolve()))
from src import *

/leonardo_scratch/fast/IscrC_MACRO/CalciumImagingDenoising/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Fourier Transform

In [19]:
x_paths = sorted(
    [_ for _ in DATASETS["synthetic"].dir.glob("*.tif") if "clean" not in _.stem],
    key=lambda _: float(_.stem.split("_")[-2].split("dBSNR")[0]),
)
cprint(*[f"rand:{' - '.join(_.stem.split('_')[1:-1])}" for _ in x_paths], sep="\n")

1Q - -5.52dBSNR
2Q - -2.51dBSNR
4Q - 0.48dBSNR
7Q - 2.88dBSNR
12Q - 5.17dBSNR
20Q - 7.31dBSNR
40Q - 10.14dBSNR
75Q - 12.56dBSNR
150Q - 14.98dBSNR
375Q - 17.56dBSNR


In [ ]:
FRAMES=300
x_path = DATASETS["synthetic"].dir / "noise_1Q_-5.52dBSNR_490x490x6000.tif"
y_path = DATASETS["synthetic"].dir / "clean_30Hz_490x490x6000.tif"
x, y = (Recording(_, max_frames=FRAMES) for _ in [x_path, y_path])
RES_DIR=Path("results/synthetic/")
RES_DIR.mkdir(exists_ok=True)

In [ ]:
fft_x = np.fft.fft(x.np, axis=0)
fft_abs_x = np.abs(fft_x)
fft_angle_x = np.angle(fft_x, deg=True)
del fft_x

In [ ]:
fft_y = np.fft.fft(y.np, axis=0)
fft_abs_y = np.abs(fft_y)
fft_angle_y = np.angle(fft_y, deg=True)
del fft_y

In [ ]:
dt = 1 / 30  # Synthetic is 30Hz
freqs = np.fft.fftfreq(x.np.shape[0], d=dt)
rfft_abs_x = fft_abs_x[freqs >= 0]
rfft_abs_y = fft_abs_y[freqs >= 0]
rfft_angle_x = fft_angle_x[freqs >= 0]
rfft_angle_y = fft_angle_y[freqs >= 0]

In [ ]:
Recording(rfft_abs_x).render("rfft_abs_x.mp4", bitrate=2000, fps=7)
Recording(rfft_abs_y).render("rfft_abs_y.mp4", bitrate=2000, fps=7)
Editor().compose(
    [{"Raw": RES_DIR / "rfft_abs_x.mp4", "DeepCAD": RES_DIR / "rfft_abs_y.mp4"}],
    RES_DIR / "rfft_abs_comb.mp4",
    bitrate=2500,
)

In [ ]:
for i in [0, 1, 2, -1]:
    cprint(f"Frequency=", f"cyan:{freqs[freqs>=0][i]:.2f}ω")
    imshow([rfft_abs_x[i], rfft_abs_y[i]])
    imshow([rfft_angle_x[i], rfft_angle_x[i]])

### The magnitude of the lowest-freq frame can be taken from RAW

In [9]:
np.save(RES_DIR/f"{FRAMES}_freq0.npy", fft_abs_x[0])

In [10]:
fft_abs = fft_abs_y
fft_abs[0] = fft_abs_x[0].copy()
del fft_abs_x, fft_angle_x
X = fft_abs_y * np.exp(1j * np.deg2rad(fft_angle_y))
astro_fused = np.fft.ifft(X, axis=0).real

In [ ]:
suffx = FRAMES
Recording(astro_fused).render(f"fft_fusion_{suffx}.mp4", fps=30)
# astro_y.render("astro_y.mp4", fps=30)
Editor().alternate(
    {"FFT Fused": RES_DIR / f"fft_fusion_{suffx}.mp4", "DeepCAD": RES_DIR / "astro_y.mp4"},
    RES_DIR / f"fft_fusion_{suffx}_alt.mp4",
    bitrate=3000,
    delta=1.5,
)
Editor().compose(
    [{"FFT Fused": RES_DIR / f"fft_fusion_{suffx}.mp4", "DeepCAD": RES_DIR / "astro_y.mp4"}],
    RES_DIR / f"fft_fusion_{suffx}_comb.mp4",
    bitrate=3500,
)

In [14]:
np.save(RES_DIR/"astro_fused.npy",astro_fused)